In [4]:
import pandas as pd

In [6]:
df_news_investing = pd.read_csv('./assignment2/GC_news_investing_05242025.csv', index_col=0)
df_news_investing.head()

,Datetime,headline,source,link
0,2025-05-23 21:35:03,Mexico stocks higher at close of trade; S&P/BM...,Investing.com,https://www.investing.com/news/stock-market-ne...
1,2025-05-23 21:25:40,Russia stocks lower at close of trade; MOEX Ru...,Investing.com,https://www.investing.com/news/stock-market-ne...
2,2025-05-23 21:25:03,Colombia stocks higher at close of trade; COLC...,Investing.com,https://www.investing.com/news/stock-market-ne...
3,2025-05-23 21:18:06,TSX closes mixed amid more Trump trade turmoil,Investing.com,https://www.investing.com/news/stock-market-ne...
4,2025-05-23 20:30:48,Canada stocks higher at close of trade; S&P/TS...,Investing.com,https://www.investing.com/news/stock-market-ne...


In [1]:
from utils.data_collection import fetch_yfinance_news_data
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import requests

In [2]:
end_time = datetime.now() - timedelta(days=3)

In [3]:
df_news = fetch_yfinance_news_data(end_time, 1000)

News Collecting...


KeyboardInterrupt



In [ ]:
df_news['article']

In [8]:
!pip install -q langchain_groq

In [14]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [37]:
def summarize_news_folder(folder_path):
    summaries = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(folder_path, filename)
            loader = TextLoader(filepath, encoding="utf-8")
            docs = loader.load()
            full_text = "\n".join([doc.page_content for doc in docs])
            headline = full_text.split('\n')[0]
            content = "\n".join(full_text.split('\n')[1:])
            result = summarization_chain.invoke({"content": content})
            summaries.append({'article': headline, 
                              'datetime': filename,
                              'comment': result.split('\n\n')[-1]})
    return summaries

In [38]:
GROQ_API_KEY = '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'

llm = ChatOllama(model="gemma3")
# llm = ChatGroq(model_name = 'llama3-70b-8192',api_key = os.environ["GROQ_API_KEY"])

In [39]:
prompt = PromptTemplate.from_template(
    """
    You are a professional financial agent. 
    You will be given an article for giving comments on the change on the gold future price.
    Summarize the following news article and given comments on the gold future market in 3-4 sentences:\n\n{article}. 
    If the article has no effect, say no effect. Otherwise, briefly tell the effect on the gold future price .
    """
)

In [40]:
summarization_chain = (
    RunnablePassthrough.assign(article=lambda x: x["content"])
    | prompt
    | llm
    | StrOutputParser()
)

In [41]:
results = summarize_news_folder('./data/new_summary_test')

In [43]:
results

[{'article': 'Morning Bid: Bond markets in the driver’s seat',
  'comment': 'No effect.'},
 {'article': 'Australia stocks higher at close of trade; S&P/ASX 200 up 0.15%',
  'comment': 'The gold futures market experienced a positive movement today, increasing by 0.94% to $3,325.89 per ounce, likely driven by broader market uncertainty as investors become increasingly cautious about stock market valuations.  The article’s mention of elevated stock market valuations and investor unease regarding finding opportunities suggests a flight to safety assets, with gold benefiting from this trend. Furthermore, the rise in gold futures coincides with positive performance in Australian stocks, particularly Perenti Global Ltd, indicating a potential correlation between risk aversion and gold’s appeal as a safe haven.'},
 {'article': 'Japan stocks higher at close of trade; Nikkei 225 up 0.42%',
  'comment': 'The significant rise in June Gold Futures (up 1.02%) indicates a positive shift in sentiment 